# understanding Chain

# Trying out GROQ

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")


In [3]:
groq_api_key = os.getenv("GROQ_API_KEY") 


In [4]:
!pip install langchain_groq

In [5]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017434CF2D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017434CF0880>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
!pip install langchain_core

Concept	Used For	                    |            Role-Based?	       |        Dynamic Variables?

HumanMessage	| Static message input from user	| ✅ Yes |	❌ No

SystemMessage	| Static system behavior instruction	| ✅ Yes |	❌ No

ChatPromptTemplate	| Creating prompts with variables and roles	| ✅ Yes	| ✅ Yes

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage
# system message is like isntructions to the model
# human message is like user input

message = [
    SystemMessage(content="Translate the following text from english to French."),
    HumanMessage(content="hello, how are you?")
]

response = model.invoke(message)
response

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 24, 'total_tokens': 51, 'completion_time': 0.049090909, 'prompt_time': 0.001327961, 'queue_time': 0.256506449, 'total_time': 0.05041887}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--53b5b413-6332-4bd1-bbc8-c12d7193aa5a-0', usage_metadata={'input_tokens': 24, 'output_tokens': 27, 'total_tokens': 51})

In [8]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(response)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ?\n'

using LCEL we can chain the componens

In [9]:
chain = model|output_parser
chain.invoke(message)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ? \n\n'

# so what we learn from all above

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

messages = [
    SystemMessage(content="Translate the following text from English to French."),
    HumanMessage(content="hello, how are you?")
]
# no need for middle code 
chain = model | StrOutputParser()
result = chain.invoke(messages)
print(result)

Bonjour, comment allez-vous ? 


Let me know if you have any other phrases you'd like me to translate! 😊



# we can also use prompttemplate , as we used in previous file

In [11]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Translate the following text from English to {language}:"),
    ("human", "{text}"),
])

In [12]:
result = prompt.invoke({
    "language":"French",
    "text" : "hello, how are you?"
})

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following text from English to French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello, how are you?', additional_kwargs={}, response_metadata={})]

In [14]:
chain= prompt|model|output_parser
result = chain.invoke({
    "language":"French",
    "text" : "hello, how are you?"
})
result

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ? \n\n'